In [16]:
%%writefile assignment31.cu
// Задание 1
// Мы умножаем каждый элемент массива на число k
// Global memory: каждый поток читает и пишет сразу в глобальную память
// Shared memory: поток сначала загружает элемент в разделямую память,  умножает, затем записывает обратно
#include <iostream>                      // Для работы с вводом/выводом
#include <cuda_runtime.h>                // Основные функции CUDA
#include <chrono>                        // Для измерения времени выполнения

using namespace std;                     // Чтобы не писать std::

// Kernel 1: Только глобальная память
__global__ void multiply_global(float *arr, float k, int n)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;   // Глобальный индекс потока

    if (idx < n)                                       // Проверка выхода за границы массива
    {
        arr[idx] = arr[idx] * k;                       // Чтение и запись напрямую в глобальную память
    }
}


// Kernel 2: Использование разделяемой памяти
__global__ void multiply_shared(float *arr, float k, int n)
{
    extern __shared__ float shmem[];                   // Объявляем динамическую shared memory

    int idx = blockIdx.x * blockDim.x + threadIdx.x;   // Глобальный индекс
    int tid = threadIdx.x;                             // Локальный индекс внутри блока

    if (idx < n)
    {
        shmem[tid] = arr[idx];                         // Загружаем данные из глобальной памяти в shared memory
        __syncthreads();                               // Синхронизация всех потоков блока

        shmem[tid] = shmem[tid] * k;                   // Умножаем в быстрой shared memory
        __syncthreads();                               // Снова синхронизация

        arr[idx] = shmem[tid];                         // Записываем результат обратно в глобальную память
    }
}

int main()                                             // Основная функция
{
    int n = 1000000;                                   // Размер массива
    size_t size = n * sizeof(float);                   // Размер в байтах
    float k = 2.5f;                                    // Множитель

    // Выделение памяти на CPU
    float *h_arr = new float[n];                       // Массив в оперативной памяти

    for (int i = 0; i < n; i++)
        h_arr[i] = 1.0f;                               // Заполняем начальными значениями


    // Выделение памяти на GPU
    float *d_arr;
    cudaMalloc(&d_arr, size);                          // Выделяем память в глобальной памяти GPU

    cudaMemcpy(d_arr, h_arr, size, cudaMemcpyHostToDevice); // Копируем данные CPU -> GPU

    int blockSize = 256;                               // Количество потоков в одном блоке
    int gridSize = (n + blockSize - 1) / blockSize;    // Количество блоков в сетке


    // Замер времени: глобальная память
    cudaDeviceSynchronize();                           // Ожидание завершения всех предыдущих операций
    auto start1 = chrono::high_resolution_clock::now();// Начало замера

    multiply_global<<<gridSize, blockSize>>>(d_arr, k, n); // Запуск kernel

    cudaDeviceSynchronize();                           // Ждем завершения kernel

    auto end1 = chrono::high_resolution_clock::now();  // Конец замера

    chrono::duration<double> time_global = end1 - start1; // Вычисление продолжительность

    // Восстанавливаем исходные данные
    cudaMemcpy(d_arr, h_arr, size, cudaMemcpyHostToDevice);

    // Замер времени: shared memory
    cudaDeviceSynchronize();
    auto start2 = chrono::high_resolution_clock::now(); // Начало замера

    multiply_shared<<<gridSize, blockSize, blockSize * sizeof(float)>>>(d_arr, k, n);
    // Третий параметр задает размер выделяемой shared memory

    cudaDeviceSynchronize();
    auto end2 = chrono::high_resolution_clock::now(); // Конец замера

    chrono::duration<double> time_shared = end2 - start2; // Вычисление продолжительность


    // Вывод результатов
    cout << "Время (Global memory): " << time_global.count() << " с" << endl;
    cout << "Время (Shared memory): " << time_shared.count() << " с" << endl;

    // Очистка памяти
    cudaFree(d_arr);                                   // Освобождаем память GPU
    delete[] h_arr;                                    // Освобождаем память CPU

    return 0;                                          // Завершения
}



Overwriting assignment31.cu


In [17]:
# Компиляция и запуск в Colab
!nvcc assignment31.cu -o assignment31
!./assignment31


Время (Global memory): 0.00751199 с
Время (Shared memory): 3.271e-06 с
